In [117]:
%tensorflow_version 2.x

In [118]:
import tensorflow as tf
# tf.version
import pandas as pd
from __future__ import absolute_import, division, unicode_literals, print_function

In [119]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [120]:
train_path = tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')

In [121]:
test_path = tf.keras.utils.get_file('iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

In [122]:
train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)

train_y = train.pop('Species')
test_y = test.pop('Species')

In [123]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [124]:
train.shape

(120, 4)

In [125]:
def input_fn(features, labels, Training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if Training:
    dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [126]:
key_feature_columns = []
for key in train.keys():
  key_feature_columns.append(tf.feature_column.numeric_column(key=key))

key_feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [127]:
classifier = tf.estimator.DNNClassifier(hidden_units=[30,10],feature_columns=key_feature_columns, n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpmfxoozng', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [128]:
classifier.train(input_fn=lambda:input_fn(train, train_y, Training=True), steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpmfxoozng/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.2973676, step = 0
INFO

In [129]:
classifier.evaluate(input_fn=lambda:input_fn(test,test_y,Training=False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-29T23:03:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmfxoozng/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.17191s
INFO:tensorflow:Finished evaluation at 2020-08-29-23:03:36
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.46666667, average_loss

{'accuracy': 0.46666667,
 'average_loss': 1.0917797,
 'global_step': 1,
 'loss': 1.0917797}

In [130]:
def input_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [131]:
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

In [132]:
print('Please type numeric values as prompted.')
for feature in features:
  valid=True
  while valid:
    val = input(feature+": ")
    if val.isdigit():
      valid = False
  predict[feature] = [float(val)]

Please type numeric values as prompted.
SepalLength: 3
SepalWidth: 4
PetalLength: 5
PetalWidth: 6


In [133]:
predict

{'PetalLength': [5.0],
 'PetalWidth': [6.0],
 'SepalLength': [3.0],
 'SepalWidth': [4.0]}

In [134]:
predictions = classifier.predict(input_fn=lambda:input_fn(predict))

In [135]:
for prediction in predictions:
  class_id = prediction['class_ids'][0]
  probability = prediction['probabilities'][class_id]
  # print('"{}" ({:.1f}%)'.format(SPECIES[class_id], probability*100))
  print('Prediction: ', SPECIES[class_id], probability)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmfxoozng/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction:  Versicolor 0.63413733
